# 4. 
##    4.1) “images/sharks.png” görüntüsünü açıp ikili bir görüntü elde ediniz. 
##    4.2)  Görüntü üzerinde daire şeklinde bir yapısal eleman kullanarak açma (opening) işlemi gerçekleştiriniz. 
##    4.3) Ardından görüntü üzerinde bağlı bileşen etiketleme algoritmasını çalıştırıp her bir etiketin farklı bir renkte gösterildiği etiket görüntüsünü oluşturunuz.
##    4.4) Algoritma neticesinde görüntüde kaç adet köpek balığı sayıldığını belirtiniz.

In [ ]:
import cv2

##    4.1) “images/sharks.png” görüntüsünü açıp ikili bir görüntü elde ediniz. 

##    4.2)  Görüntü üzerinde daire şeklinde bir yapısal eleman kullanarak açma (opening) işlemi gerçekleştiriniz. 


##    4.3) Ardından görüntü üzerinde bağlı bileşen etiketleme algoritmasını çalıştırıp her bir etiketin farklı bir renkte gösterildiği etiket görüntüsünü oluşturunuz.


##    4.4) Algoritma neticesinde görüntüde kaç adet köpek balığı sayıldığını belirtiniz.